In [1]:
import pandas as pd
import numpy as np
import os
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

import dask.dataframe as dd
from tqdm import trange
import lightgbm, joblib

# 測試集載入

In [57]:
test = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_22_24dt.csv")

In [58]:
chid_set = set(range(10000000, 10500000)) - set(test.chid)
chid_set = list(chid_set)
len(chid_set)

31207

# 訓練集載入

In [59]:
train_1 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(13, 15)).reset_index(drop=True)
train_2 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(14, 16)).reset_index(drop=True)
train_3 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(15, 17)).reset_index(drop=True)
train_4 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(16, 18)).reset_index(drop=True)
train_5 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(17, 19)).reset_index(drop=True)
train_6 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(18, 20)).reset_index(drop=True)
train_7 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(19, 21)).reset_index(drop=True)
train_8 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(20, 22)).reset_index(drop=True)
train_9 = pd.read_csv("./訓練資料集/model/0103/data/traindata_at_{}_{}dt.csv".format(21, 23)).reset_index(drop=True)

In [60]:
for i in trange(len(chid_set)):
    id_ = chid_set[i]

    if id_ in list(train_9.chid):
        df_append = train_9[train_9.chid==id_]
        test = test.append(df_append.drop(columns=['y_tag']))
        train_9 = train_9.drop(df_append.index)
    
    elif id_ in list(train_8.chid):
        df_append = train_8[train_8.chid==id_]
        test = test.append(df_append.drop(columns=['y_tag']))
        train_8 = train_8.drop(df_append.index)
        
    elif id_ in list(train_7.chid):
        df_append = train_7[train_7.chid==id_]
        test = test.append(df_append.drop(columns=['y_tag']))
        train_7 = train_7.drop(df_append.index)
    
    elif id_ in list(train_6.chid):
        df_append = train_6[train_6.chid==id_]
        test = test.append(df_append.drop(columns=['y_tag']))
        train_6 = train_6.drop(df_append.index)
    
    elif id_ in list(train_5.chid):
        df_append = train_5[train_5.chid==id_]
        test = test.append(df_append.drop(columns=['y_tag']))
        train_5 = train_5.drop(df_append.index)
                                
    elif id_ in list(train_4.chid):
        df_append = train_4[train_4.chid==id_]
        test = test.append(df_append.drop(columns=['y_tag']))
        train_4 = train_4.drop(df_append.index)
        
    elif id_ in list(train_3.chid):
        df_append = train_3[train_3.chid==id_]
        test = test.append(df_append.drop(columns=['y_tag']))
        train_3 = train_3.drop(df_append.index)
        
    elif id_ in list(train_2.chid):
        df_append = train_2[train_2.chid==id_]
        test = test.append(df_append.drop(columns=['y_tag']))
        train_2 = train_2.drop(df_append.index)
        
    elif id_ in list(train_1.chid):
        df_append = train_1[train_1.chid==id_]
        test = test.append(df_append.drop(columns=['y_tag']))
        train_1 = train_1.drop(df_append.index)
        
    else:
        a = pd.DataFrame([id_] + [0]*34).transpose()
        a.columns = ['chid', '2_cnt', '6_cnt', '10_cnt', '12_cnt', '13_cnt', '15_cnt',
                     '18_cnt', '19_cnt', '21_cnt', '22_cnt', '25_cnt', '26_cnt', '36_cnt',
                     '37_cnt', '39_cnt', '48_cnt', '2_amt', '6_amt', '10_amt', '12_amt',
                     '13_amt', '15_amt', '18_amt', '19_amt', '21_amt', '22_amt', '25_amt',
                     '26_amt', '36_amt', '37_amt', '39_amt', '48_amt', 'txn_amt', 'txn_cnt']
        test = test.append(a)

100%|██████████████████████████████████████████████████████████████████████████| 31207/31207 [3:10:31<00:00,  2.73it/s]


In [61]:
test.shape

(500000, 35)

In [62]:
test = test.sort_values(by=['chid']).reset_index(drop=True)
test.to_csv("./訓練資料集/model/0105/testing_data_remedy.csv", index=False)

In [67]:
test=''

In [68]:
train_1.head()

,chid,2_cnt,6_cnt,10_cnt,12_cnt,13_cnt,15_cnt,18_cnt,19_cnt,21_cnt,...,22_amt,25_amt,26_amt,36_amt,37_amt,39_amt,48_amt,txn_amt,txn_cnt,y_tag
0,10000000,0.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.0,53358.815509,12.0,10
1,10000001,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,2136.2535,5972.879875,0.0,0.0,28310.733048,8.0,39
2,10000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8554.030804,0.0,0.0,0.0000,0.000000,0.0,0.0,8554.030804,1.0,0
3,10000004,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.0,15271.878083,2.0,37
4,10000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0000,6945.246505,0.0,0.0,6945.246505,2.0,0


In [63]:
df = pd.concat([train_1, train_2, train_3 , train_4, train_5, train_6, train_7, train_8, train_9]).reset_index(drop=True)

In [64]:
df = df[df['y_tag']>0]

In [65]:
df.shape

(3339148, 36)

In [66]:
df.to_csv("./訓練資料集/model/0105/training_data_remedy_removed.csv", index=False)

# 訓練模型

In [108]:
#train = pd.read_csv("./訓練資料集/model/0105/training_data_remedy_removed.csv")
train = pd.read_csv("./訓練資料集/model/0105/training_data_add_others.csv")
test = pd.read_csv("./訓練資料集/model/0105/testing_data_remedy.csv")
test = pd.read_csv("./訓練資料集/model/0105/testing_data_add_others.csv")

In [109]:
train['y_tag'].value_counts()

37    825248
10    440857
2     353637
36    313028
15    285846
48    248639
12    153808
19    147654
39    102952
6      95320
18     76735
13     73675
26     71247
25     53368
21     51944
22     45190
Name: y_tag, dtype: int64

In [115]:
import lightgbm
model = lightgbm.LGBMClassifier(max_depth=5, learning_rate=0.05, min_child_samples=100)
X = train.drop(columns=['chid', 'y_tag'])
y = train['y_tag']
model.fit(X, y)

LGBMClassifier(learning_rate=0.05, max_depth=5, min_child_samples=100)

In [116]:
# Save model
import joblib
joblib.dump(model, './訓練資料集/model/0105/lightgbm_model_with_balanced_add_others.pkl')

['./訓練資料集/model/0105/lightgbm_model_with_balanced_add_others.pkl']

In [117]:
y_proba = model.predict_proba(test.drop(columns=['chid']))  # 

In [118]:
y_proba.shape

(500000, 16)

In [119]:
tag_unique = [2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]
result = []
for i in trange(y_proba.shape[0]):
    proba_ls = [(proba, tag) for proba, tag in zip(y_proba[i], tag_unique)]
    proba_ls = sorted(proba_ls, reverse=True)
    pred_tag_ls = [tag for proba, tag in proba_ls[:3]]
    result.append(pred_tag_ls)

100%|███████████████████████████████████████████████████████████████████████| 500000/500000 [00:08<00:00, 58849.58it/s]


In [120]:
result = pd.DataFrame(result, columns=['top1', 'top2', 'top3'])
result['chid'] = list(test.reset_index().chid)
result = result[['chid', 'top1', 'top2', 'top3']]

In [121]:
result.top1.value_counts()

37    168886
10     72175
2      59144
36     46852
48     41820
15     36467
12     19098
6      11078
13     10105
19      9086
39      6061
22      6043
26      5104
18      4045
25      3382
21       654
Name: top1, dtype: int64

In [122]:
result.top2.value_counts()

37    100013
2      94568
15     92579
10     52589
36     34241
48     28974
19     20350
12     17158
39     14728
13      9629
18      7119
26      6976
6       6709
22      5586
25      5460
21      3321
Name: top2, dtype: int64

In [123]:
result.top3.value_counts()

15    131949
2     119230
37     56625
10     39604
19     28962
48     23936
36     23782
39     20314
12     13245
18      8695
13      7604
26      6446
6       5343
25      5201
21      4888
22      4176
Name: top3, dtype: int64

In [124]:
result.head(3)

,chid,top1,top2,top3
0,10000000,10,48,37
1,10000001,48,10,37
2,10000002,22,37,2


In [125]:
result.shape

(500000, 4)

In [126]:
result = result.astype("str")
result.to_csv('./訓練資料集/model/0105/將tag攤開成32維(增加訓練資料)_AddOthers.csv', index=False)